In [ ]:
import sys
import math

class TwoOptSolver:
    def __init__(self, filename):

        distance_matrix = []
        euc_coordinates = []
        explicit = True

        with open(filename, 'r') as f:
            lines = f.readlines()
            read_distances = False  

            for line in lines: 
                line = line.strip()

                if line.startswith('EOF'):
                    break
                elif line.startswith('EDGE_WEIGHT_TYPE'):
                    if 'EXPLICIT' in line:
                        explicit = True
                    else:
                        explicit = False
                elif line.startswith('EDGE_WEIGHT_SECTION') or line.startswith('NODE_COORD_SECTION'):
                    read_distances = True
                elif explicit and read_distances:
                    elements = line.split()
                    distance_matrix.append([int(e) for e in elements])
                elif explicit == False and read_distances: 
                    elements = line.split()
                    x = int(elements[1])
                    y = int(elements[2])
                    euc_coordinates.append((x, y))

        if explicit:
            self.distance_matrix = distance_matrix
            self.n = len(distance_matrix)
            self.explicit = True
        else:
            self.coordinates = euc_coordinates
            self.n = len(euc_coordinates)
            self.explicit = False

    def get_cost(self, i, j):
        if self.explicit:
            return self.distance_matrix[i][j]    
        else:
            x1, y1 = self.coordinates[i]
            x2, y2 = self.coordinates[j]
            return math.ceil(((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5)
    
    def calc_path_length(self):
        path_cost = 0

        for i in range((self.n - 1)):
            vertex_i = self.best_route[i]
            vertex_j = self.best_route[i + 1]
            cost_ij = self.get_cost(vertex_i, vertex_j)
            path_cost = path_cost + cost_ij

        # n -> 0
        vertex_i = self.best_route[self.n - 1]
        vertex_j = self.best_route[0]
        cost_ij = self.get_cost(vertex_i, vertex_j)
        path_cost = path_cost + cost_ij        

        return path_cost
    
    def swap_edges(self, index_i, index_j):
        index_i = index_i + 1

        while index_i < index_j: 
            tmp_vertex = self.best_route[index_i]
            self.best_route[index_i] = self.best_route[index_j]
            self.best_route[index_j] = tmp_vertex
            index_i = index_i + 1
            index_j = index_j - 1


    def two_opt(self):
        
        # route
        self.best_route = list(range(self.n))
        self.best_length = self.calc_path_length()
        improvement_found = True

        while improvement_found == True:
            improvement_found = False

            for i in range((self.n - 1)):

                for j in range((i + 2), self.n):

                    j_1 = (j + 1) % self.n

                    vertex_i = self.best_route[i]
                    vertex_i1 = self.best_route[i + 1]
                    vertex_j = self.best_route[j]
                    vertex_j1 = self.best_route[j_1]


                    r_i_i1 = self.get_cost(vertex_i, vertex_i1)
                    r_j_j1 = self.get_cost(vertex_j, vertex_j1)
                    r_i_j = self.get_cost(vertex_i, vertex_j)
                    r_i1_j1 = self.get_cost(vertex_i1, vertex_j1)

                    minus = r_i_i1 + r_j_j1
                    plus = r_i_j + r_i1_j1

                    len_delta = plus - minus

                    if len_delta < 0: 
                        # better route found by going i -> j -> ... -> j+1 -> i+1 -> ...
                        improvement_found = True
                        self.best_length = self.best_length + len_delta
                        self.swap_edges(i, j)

        return self.best_length, self.best_route

In [11]:
#filename = "../tsplib_converted_instances/03_large/bier127_converted.tsp"
filename = "../tsplib_instances_project/02_medium/eil76.tsp"

# Solve using 2-Opt 
two_opt_solver = TwoOptSolver(filename)
to_cost, to_path = two_opt_solver.two_opt()

print("\nTwo-Opt-Solver - Minimum Cost:", to_cost)
print("\nPath: ", to_path)


Two-Opt-Solver - Minimum Cost: 631

Path:  [0, 42, 40, 41, 63, 21, 27, 60, 20, 46, 35, 68, 70, 59, 69, 19, 36, 56, 14, 4, 28, 47, 29, 73, 1, 67, 74, 75, 3, 44, 33, 51, 26, 12, 53, 18, 34, 6, 7, 45, 66, 25, 11, 57, 52, 13, 58, 10, 65, 64, 37, 9, 30, 71, 38, 24, 54, 49, 17, 23, 48, 55, 22, 62, 15, 2, 43, 31, 8, 39, 16, 50, 5, 32, 72, 61]
